# Advent of Code 2023

## Contents
- [Day 1](#day-1)
- [Day 2](#day-2)
- [Day 3](#day-3)
- [Day 4](#day-4)
- [Day 5](#day-5)
- [Day 6](#day-6)
- [Day 7](#day-7)
- [Day 8](#day-8)
- [Day 9](#day-9)
- [Day 10](#day-10)
- [Day 11](#day-11)
- [Day 12](#day-12)

## Boilerplate

In [54]:
# SETUP #
import sys
import math
import operator
import numpy as np
from itertools import compress, combinations
from functools import reduce, cmp_to_key, cache

# TEST #
def test(test, solution):
    if test != solution:
        print(f"Test failed. Expected {solution}, but got {test}.")
    else: print("Test success!")

In [55]:
# FILE READING - Run *after* day config #
with open(f"inputs/{DAY}.txt", mode="rt") as f:
    PUZZLE_INPUT = f.read()

with open(f"test_inputs/{DAY}.txt", mode="rt") as f:
    TEST_INPUT = f.read()

## Day 1

[Link to puzzle](https://adventofcode.com/2023/day/1)

In [101]:
# DAY CONFIG #
DAY = "1"
TEST_SOLUTION_PART_1 = 142
TEST_SOLUTION_PART_2 = 281

In [104]:
%%time
# DAY 1 #
def run(part,i):
    
    # Part 2 only
    if part == 2:
        number_names = [
            ("one", "one1one"),
            ("two", "two2two"),
            ("three", "three3three"),
            ("four", "four4four"),
            ("five", "five5five"),
            ("six", "six6six"),
            ("seven", "seven7seven"),
            ("eight", "eight8eight"),
            ("nine", "nine9nine"),
        ]
        for pair in number_names:
            i = i.replace(pair[0], pair[1])

    total = 0
    # For each line, combine the first and last digits and add to total
    for line in i.splitlines():
        digits = ''.join(c for c in line if c.isdigit())
        total += int(digits[0] + digits[-1])
    return total

# Run test
test(run(1,"1abc2\npqr3stu8vwx\na1b2c3d4e5f\ntreb7uchet"), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 54951
Part 2: 55218
CPU times: user 7.99 ms, sys: 771 µs, total: 8.76 ms
Wall time: 8.25 ms


## Day 2

[Link to puzzle](https://adventofcode.com/2023/day/2)

In [105]:
# DAY CONFIG #
DAY = "2"
TEST_SOLUTION_PART_1 = 8
TEST_SOLUTION_PART_2 = 2286

In [108]:
%%time
# DAY 2 #
def run(part,i):
    
    def clean_input(input):
        # Parse the input into the following format:
        # {'1': {'red': 4, 'green': 2, 'blue': 6}, '2': {'red': 1, 'green': 3, 'blue': 4}...
        clean = {}

        # Input line format -> Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
        for line in input.splitlines():
            # Break into "Game X" and game data (list of hands), then isolate game ID
            id, game = line.split(": ")
            id = id.split(" ")[1]
            
            clean[id] = {"red": 0, "green": 0, "blue": 0}

            for round in game.split("; "):
                for r in round.split(", "):
                    amount, colour = r.split(" ")
                    # Update the value for a colour only if it is higher than the existing value
                    clean[id][colour] = max(clean[id][colour], int(amount))

        return clean

    def validate(id,game):
        # Return game ID for valid games, 0 for invalid
        max_values = {"red": 12, "green": 13, "blue": 14}

        for colour, value in game.items():
            if value > max_values[colour]:
                return 0
            
        return int(id)

    def get_power(game):
        return math.prod(game.values())

    data = clean_input(i)
    
    total = 0
    total_power = 0

    for id, game in data.items():
        total += validate(id, game)
        total_power += get_power(game)

    return total if (part == 1) else total_power

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 2449
Part 2: 63981
CPU times: user 3.75 ms, sys: 101 µs, total: 3.86 ms
Wall time: 3.94 ms


## Day 3

[Link to puzzle](https://adventofcode.com/2023/day/3)

In [109]:
# DAY CONFIG #
DAY = "3"
TEST_SOLUTION_PART_1 = 4361
TEST_SOLUTION_PART_2 = 467835

In [111]:
%%time
# DAY 3 #
def run(part,i):
    # Convert input to 2D matrix
    i = list(map(list, i.splitlines()))

    checked = [[] for _ in i]
    parts = []
    gears = []

    def find_adjacent_numbers(row, column) -> ([int], int):
        adjacent_count = 0
        numbers = []
        gear_power = 0

        # Check row and column before and after
        for r in range(row-1, row+2):
            for c in range(column-1, column+2):
                if number := check_digit(r,c): 
                    numbers.append(number)
                    adjacent_count += 1

        if i[row][column] == '*' and adjacent_count == 2:
            gear_power = math.prod(numbers)
        
        return (numbers, gear_power)

    def check_digit(row, column) -> int:
        if row < 0 or row >= len(i) or column < 0 or column >= len(i[row]):
            return # out of bounds
        
        if i[row][column].isdigit() and column not in checked[row]:
            # Walk back to start of digit and save it
            number = ''
            while column > 0 and i[row][column-1].isdigit():
                column -= 1
                
            while column < len(i[row]) and i[row][column].isdigit():
                number += i[row][column]
                checked[row].append(column)
                column += 1

            return int(number)

    for row in range(len(i)):
        for column in range(len(i[row])):
            # Check for symbol, find numbers near it
            if not i[row][column].isdigit() and i[row][column] != '.':
                numbers, score = find_adjacent_numbers(row, column)
                parts.extend(numbers)
                gears.append(score)

    return sum(parts) if part == 1 else sum(gears)

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 556057
Part 2: 82824352
CPU times: user 17.2 ms, sys: 1.43 ms, total: 18.7 ms
Wall time: 17.6 ms


## Day 4

[Link to puzzle](https://adventofcode.com/2023/day/4)

In [112]:
# DAY CONFIG #
DAY = "4"
TEST_SOLUTION_PART_1 = 13
TEST_SOLUTION_PART_2 = 30

In [115]:
%%time
# DAY 4 #

def run(part,i):
    
    def clean_input(input):
        # Format -> Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
        # Return generator of id/wins tuples
        for line in input.splitlines():
            id, data = line.split(": ")
            id = int(id.split(" ")[-1])
            winners, card = (set(x.split()) for x in data.split(" | "))
            wins = len(winners & card)
            yield id, wins
    
    score = 0
    copies = [1] * len(i.splitlines()) # 1 copy of each card to start

    for id, wins in clean_input(i):
        score += 1 << (wins - 1) if wins > 0 else 0 # bit shift - same as 2^(wins-1)
        for x in range(wins):
            copies[id+x] += copies[id-1]

    return score if part == 1 else sum(copies)

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 28750
Part 2: 10212704
CPU times: user 6.17 ms, sys: 535 µs, total: 6.71 ms
Wall time: 6.52 ms


## Day 5

[Link to puzzle](https://adventofcode.com/2023/day/5)

In [116]:
# DAY CONFIG #
DAY = "5"
TEST_SOLUTION_PART_1 = 35
TEST_SOLUTION_PART_2 = 46

In [119]:
%%time
# DAY 5 #

def run(part,i):

    def clean_input(input):
        groups = input.split("\n\n")
        seeds = groups[0].split()[1:]
 
        mappings = []
        for map in groups[1:]:
            section = []
            for line in map.splitlines()[1:]:
                start_dest, start_source, length = [int(val) for val in line.split()]
                section.append((range(start_source, start_source + length), start_dest))
            mappings.append(section)

        return seeds, mappings

    def map_value(value, section):
        for range, destination in section:
            if value in range:
                return destination + (value - range.start)
        return value

    def check_ranges(ranges_to_check, mapping):
        new_ranges = []

        for current_range in ranges_to_check:
            for map_range, next_start in mapping:
                intersection = range(max(map_range.start, current_range.start), min(map_range.stop, current_range.stop))
                
                if intersection: # if there is a valid intersection
                    offset = next_start - map_range.start
                    new_ranges.append(range(intersection.start + offset, intersection.stop + offset))

                    # Update ranges to check based on the intersection
                    if intersection.start > current_range.start:
                        ranges_to_check.append(range(current_range.start, intersection.start))

                    if intersection.stop < current_range.stop:
                        ranges_to_check.append(range(intersection.stop, current_range.stop))

                    break
            else:
                new_ranges.append(current_range)

        return new_ranges
    
    # Main functions #
    
    def part1(): 
        locations = []   
        seeds, mappings = clean_input(i)
        for seed in seeds:
            for section in mappings:
                seed = map_value(int(seed), section)
            locations.append(seed)   
        return min(locations)

    def part2():
        seeds, mappings = clean_input(i)
        seed_ranges = [
            range(int(seeds[i]), int(seeds[i + 1]) + int(seeds[i])) 
            for i in range(0, len(seeds), 2)
        ]

        lowest = float('inf') # really big number
        for sr in seed_ranges:
            ranges = [sr]

            for mapping in mappings:
                ranges = check_ranges(ranges, mapping)

            lowest = min(lowest, min(r.start for r in ranges))

        return lowest
                
    return part1() if part == 1 else part2()

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 313045984
Part 2: 20283860
CPU times: user 5.69 ms, sys: 549 µs, total: 6.23 ms
Wall time: 5.85 ms


## Day 6

[Link to puzzle](https://adventofcode.com/2023/day/6)

In [137]:
# DAY CONFIG #
DAY = "6"
TEST_SOLUTION_PART_1 = 288
TEST_SOLUTION_PART_2 = 71503

In [184]:
%%time
# DAY 6 #

def run(part,i):
    
    def clean_input(input, combined=True):
        lines = input.splitlines()
        if combined:
            # Tuple of (time, distance)
            result = (int(''.join(line.split()[1:])) for line in lines)
        else:
            # List of (time, distance) for each race
            result = list(zip(map(int, lines[0].split()[1:]), map(int, lines[1].split()[1:])))
        return result

    def winning_cases_brute(time, distance):
        # Simple solution, ~5s for part 2
        # True/False for win/lose
        wins = [speed * (time - speed) > distance for speed in range(time)]
        return sum(wins)

    def winning_cases_quadratic(time, distance):
        # Quadratic equation time!
        # a = 1, b = -time, c = distance
        sqrt_discriminant = math.sqrt(time**2 - 4*distance)
        upper, lower = (time + sqrt_discriminant) / 2, (time - sqrt_discriminant) / 2
        return math.ceil(upper) - math.floor(lower) -1
    
    # Main functions #

    def part1():
        # Multiply the winning cases for each race
        races = clean_input(i, False)
        return reduce(operator.mul, (winning_cases_quadratic(*race) for race in races), 1)

    def part2():
        # Get number of winning cases for single race
        time, distance = clean_input(i)
        return winning_cases_quadratic(time, distance)
                
    return part1() if part == 1 else part2()

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 252000
Part 2: 36992486
CPU times: user 403 µs, sys: 66 µs, total: 469 µs
Wall time: 449 µs


## Day 7

[Link to puzzle](https://adventofcode.com/2023/day/7)

In [226]:
# DAY CONFIG #
DAY = "7"
TEST_SOLUTION_PART_1 = 6440
TEST_SOLUTION_PART_2 = 5905

In [265]:
%%time
# DAY 7 #

def run(part,i):
    
    def clean_input(input):
        # [(hand, bid)...]
        return [line.split() for line in input.strip().split('\n')]

    values = {
        "A": 14,
        "K": 13,
        "Q": 12,
        "J": 11,
        "T": 10,
        "9": 9,
        "8": 8,
        "7": 7,
        "6": 6,
        "5": 5,
        "4": 4,
        "3": 3,
        "2": 2
    }

    def get_points(hand):
        # 6 Five of a kind, where all five cards have the same label: AAAAA
        # 5 Four of a kind, where four cards have the same label and one card has a different label: AA8AA
        # 4 Full house, where three cards have the same label, and the remaining two cards share a different label: 23332
        # 3 Three of a kind, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98
        # 2 Two pair, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432
        # 1 One pair, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4
        # 0 High card, where all cards' labels are distinct: 23456
        cards = [card for card in hand]
        counts = {card_type: cards.count(card_type) for card_type in set(cards)}

        if 5 in counts.values():
            return 6  # Five of a kind
        elif 4 in counts.values():
            return 5  # Four of a kind
        elif sorted(counts.values()) == [2, 3]:
            return 4  # Full house
        elif 3 in counts.values():
            return 3  # Three of a kind
        elif sorted(counts.values()) == [1, 2, 2]:
            return 2  # Two pair
        elif sorted(counts.values()) == [1, 1, 1, 2]:
            return 1  # One pair
        else:
            return 0

    def compare_hands(left, right):
        left, right = left[0], right[0]

        if (part == 2):
            hand_type_diff = max_joker(left) - max_joker(right)
        else:
            hand_type_diff = get_points(left) - get_points(right)
        if hand_type_diff != 0:
            return hand_type_diff

        # Same point value - move to tie breaker
        # Compare the values of the first non-matching character
        return next((values[a] - values[b] for a, b in zip(left, right) if a != b), 0)
    
    def max_joker(hand):
        max_points = 0
        for v in values.keys(): # check point result of replacing with each value
            test_hand = hand.replace("J",v)
            if get_points(test_hand) > max_points:
                max_points = get_points(test_hand)
        return max_points
    
    data = clean_input(i)

    winnings = 0
    if part == 2: values["J"] = 1 # in part 2, J is less valuable
    for index, hand in enumerate(sorted(data, key=cmp_to_key(compare_hands))):
        winnings += int(hand[1]) * (index+1) # bet * rank of hand
                
    return winnings

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 252656917
Part 2: 253499763
CPU times: user 499 ms, sys: 1.72 ms, total: 500 ms
Wall time: 505 ms


## Day 8

[Link to puzzle](https://adventofcode.com/2023/day/8)

In [59]:
# DAY CONFIG #
DAY = "8"
TEST_SOLUTION_PART_1 = 6
TEST_SOLUTION_PART_2 = 6

In [64]:
%%time
# DAY 8 #

def run(part,i):
    
    def clean_input(input):
        clean = {}
        instructions, data = input.split("\n\n")

        for line in data.splitlines():
            key, values = line.split(' = ')
            left, right = values.strip()[1:-1].split(", ")
            clean[key] = (left, right)

        return instructions, clean

    def count_steps(start, destination_check, instructions, mapping):
        count = 0
        current_location = start
        while True:
            for step in instructions:
                count += 1
                if step == 'L':
                    current_location = mapping[current_location][0]
                if step == 'R':
                    current_location = mapping[current_location][1]
                if destination_check(current_location):
                    return count

    def part1():
        instructions, mapping = clean_input(i)
        return count_steps('AAA', lambda x: x == 'ZZZ', instructions, mapping)
    
    def part2():
        instructions, mapping = clean_input(i)
        starting_locations = [key for key in mapping if key.endswith('A')]
        counts = []

        # Find the # of steps for each location to get from A -> Z
        for location in starting_locations:
            counts.append(count_steps(location, lambda x: x.endswith('Z'), instructions, mapping))
        
        # LCM of all path lengths will be the first time that all paths end on Z simultaneously
        return(math.lcm(*counts))
                
    return part1() if part == 1 else part2()

# Run test
test(run(1,'LLR\n\nAAA = (BBB, BBB)\nBBB = (AAA, ZZZ)\nZZZ = (ZZZ, ZZZ)'), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 20569
Part 2: 21366921060721
CPU times: user 47.8 ms, sys: 1.43 ms, total: 49.2 ms
Wall time: 48.2 ms


## Day 9

[Link to puzzle](https://adventofcode.com/2023/day/9)

In [65]:
# DAY CONFIG #
DAY = "9"
TEST_SOLUTION_PART_1 = 114
TEST_SOLUTION_PART_2 = 2

In [67]:
%%time
# DAY 9 #

def run(part,i):
    
    def clean_input(input):
        clean = [[int(i) for i in line.split()] for line in input.splitlines()]
        return clean

    def extrapolate(sequence):
        # Calculate differences between consecutive elements in the sequence
        diffs = [(b - a) for a, b in zip(sequence, sequence[1:])]

        # Recursively extrapolate the differences and add the result to the last element
        # If all zeroes, return 0
        return sequence[-1] + extrapolate(diffs) if sequence else 0

    def part1():
        input = clean_input(i)
        return sum(extrapolate(line) for line in input)
    
    def part2():
        input = clean_input(i)
        # Reverse the list and extrapolate to extrapolate "backwards"
        return sum(extrapolate(line[::-1]) for line in input)
                
    return part1() if part == 1 else part2()

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 1921197370
Part 2: 1124
CPU times: user 14.9 ms, sys: 825 µs, total: 15.7 ms
Wall time: 14.9 ms


## Day 10

[Link to puzzle](https://adventofcode.com/2023/day/10)

In [2]:
# DAY CONFIG #
DAY = "10"
TEST_SOLUTION_PART_1 = 8
TEST_SOLUTION_PART_2 = 10

In [10]:
%%time
# DAY 10 #

def run(part,i):
    
    def clean_input(input):
        # Convert to prettier characters (─│┌ ┐└ ┘) for better visualisations
        i = input.translate(str.maketrans("-|F7LJ", "─│┌┐└┘"))
        # Use np array so `array[(x,y)]` syntax works
        return np.array(list(map(list, i.splitlines())))
    
    UP = (-1, 0)
    DOWN = (1, 0)
    LEFT = (0, -1)
    RIGHT = (0, 1)

    moves = {
        ('│', DOWN): DOWN,
        ('│', UP): UP,
        ('─', RIGHT): RIGHT,
        ('─', LEFT): LEFT,
        ('└', LEFT): UP,
        ('└', DOWN): RIGHT,
        ('┘', RIGHT): UP,
        ('┘', DOWN): LEFT,
        ('┐', RIGHT): DOWN,
        ('┐', UP): LEFT,
        ('┌', LEFT): DOWN,
        ('┌', UP): RIGHT
    }
    
    def find_start(i):
        # This is way too much effort to make it work for any input. Hardcoding is easier.
        directions = set()
        for r, row in enumerate(i):
            for c, character in enumerate(row):
                if character == 'S':
                    # Find the 'S' and the valid pipe directions
                    position = (r, c)
                    if i[r-1, c] in "│┌┐":
                        directions.add(UP)
                    if i[r+1, c] in "│└┘":
                        directions.add(DOWN)
                    if i[r, c-1] in "─┌└":
                        directions.add(LEFT)
                    if i[r, c+1] in "─┐┘":
                        directions.add(RIGHT)

        # What shape 'S' should be to complete the loop
        if {UP, DOWN}.issubset(directions):
            shape = '│'
        if {LEFT, RIGHT}.issubset(directions):
            shape = '─'
        if {UP, LEFT}.issubset(directions):
            shape = '┘'
        if {UP, RIGHT}.issubset(directions):
            shape = '└'
        if {DOWN, LEFT}.issubset(directions):
            shape = '┐'
        if {DOWN, RIGHT}.issubset(directions):
            shape = '┌'
            
        return (position, directions.pop(), shape)

    def walk_loop(start, dir, mapping, path):
        current = tuple(map(sum, zip(start, dir))) # First step
        while current != start:
            # Adjust coordinates to next position based on direction
            path.append(current)
            dir = moves[mapping[current], dir]
            current = tuple(map(sum, zip(current, dir)))
        return path

    # Cleaned up a lot with help from this hungover guy quoting the Matrix:
    # https://github.com/AlbertVeli/AdventOfCode/blob/master/2023/10/day10.py  
    def map_contained_area(path, i): 
        for r, row in enumerate(i):
            for c, column in enumerate(row):
                if (r, c) not in path:
                    i[r, c] = '.'

        # https://gamedev.stackexchange.com/questions/141460/how-can-i-fill-the-interior-of-a-closed-loop-on-a-tile-map
        area = 0
        for row in i:
            inside = False
            for c in row:
                if c in "│┘└":
                    inside = not inside
                elif c == '.' and inside:
                    area += 1

        return area
    
    # Start here #
                
    input = clean_input(i)
    start, direction, start_shape = find_start(input) # Alternatively, ctrl-F to find S and hard code these
    input[start] = start_shape # Replace S with appropriate shape
    path = walk_loop(start, direction, input, [start]) 

    return len(path)//2 if part == 1 else map_contained_area(path, input)

# Run test
test(run(1,"7-F7-\n.FJ|7\nSJLL7\n|F--J\nLJ.LJ"), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 6828
Part 2: 459
CPU times: user 2.96 s, sys: 2.05 ms, total: 2.96 s
Wall time: 2.97 s


## Day 11

[Link to puzzle](https://adventofcode.com/2023/day/11)

In [71]:
# DAY CONFIG #
DAY = "11"
TEST_SOLUTION_PART_1 = 374
TEST_SOLUTION_PART_2 = 8410

In [101]:
%%time
# DAY 11 #

def run(part,i,test=False):

    def map_universe(input):
        universe = np.array(list(map(list, i.splitlines())))

        # Find empty rows and columns
        empty_rows = np.all(universe == '.', axis=1)
        empty_columns = np.all(universe == '.', axis=0)

        # Original part 1 solution to actually expand universe:
        # universe = np.insert(universe, np.where(empty_rows)[0], '.', axis=0)
        # universe = np.insert(universe, np.where(empty_columns)[0], '.', axis=1)

        # Using set instead of list here brought time from ~10s to ~1s 
        return universe, set(np.where(empty_rows)[0]), set(np.where(empty_columns)[0])
    
    def distance(p1, p2, double_rows, double_cols, factor):
        # Manhattan distance -> abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])
        return (
            sum(factor if x in double_cols else 1 for x in range(*sorted([p1[0], p2[0]]))) +
            sum(factor if y in double_rows else 1 for y in range(*sorted([p1[1], p2[1]]))))
    
    # Start here #

    # Double space for part 1, 100x for part 2 test, 1000000x for part 2
    factor = 2 if part == 1 else (100 if test else 1000000)

    universe, double_rows, double_cols = map_universe(i)
    galaxies = []
    for r, row in enumerate(universe):
        for c, character in enumerate(row):
            if character == '#':
                galaxies.append((c,r))

    distances = []
    for combo in combinations(galaxies, 2):
        distances.append(distance(combo[0], combo[1], double_rows, double_cols, factor))

    return sum(distances)

# Run test
test(run(1,TEST_INPUT,True), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT,True), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 9214785
Part 2: 613686987427
CPU times: user 1.38 s, sys: 3.67 ms, total: 1.39 s
Wall time: 1.39 s


## Day 12

[Link to puzzle](https://adventofcode.com/2023/day/12)

In [2]:
# DAY CONFIG #
DAY = "12"
TEST_SOLUTION_PART_1 = 21
TEST_SOLUTION_PART_2 = 525152

In [50]:
%%time
# DAY 12 #

def run(part,i):

    def clean_input(input, unfold=False):
        # list of (record, numbers), e.g. ('???.###', [1,1,3])
        clean = [
            (record, [int(x) for x in numbers.split(',')])
            for record, numbers in (line.split() for line in input.splitlines())
        ]

        if unfold:
            return [
                ('?'.join([record] * 5), numbers * 5)
                for record, numbers in clean
            ]
        return clean

    @cache # Very important to cache - takes at least a couple minutes without (maybe more)
    def count_valid_combinations(record, numbers, count=0):
        # Base case: end of the record, check validity
        if not record: 
            if len(numbers) == 1 and numbers[0] == count: 
                return 1
            elif len(numbers) == 0 and count == 0:
                return 1
            else:
                return 0
        
        # Remove first character from the record and first number from the number list
        c, record = record[0], ''.join(record[1:])
        n, *remaining_numbers = numbers or [0]
        
        match c:
            case '?':
                # Try combinations for both operational and damaged
                return count_valid_combinations('#'+record, numbers, count) + \
                    count_valid_combinations('.'+record, numbers, count)
            case '#':
                return 0 if count > n else count_valid_combinations(record, numbers, count+1)
            case '.':
                if count == 0:
                    return count_valid_combinations(record, numbers)
                elif count == n:
                    return count_valid_combinations(record, tuple(remaining_numbers))
                else:
                    return 0

    # Start here #
    unfold = False if part == 1 else True
    input = clean_input(i, unfold)

    solutions = 0
    for record, numbers in input:
        solutions += count_valid_combinations(record, tuple(numbers)) # Tuple so we can use @cache

    return solutions

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 7857
Part 2: 28606137449920
CPU times: user 2.98 s, sys: 80.4 ms, total: 3.06 s
Wall time: 3.07 s


## Day 13

[Link to puzzle](https://adventofcode.com/2023/day/13)

In [52]:
# DAY CONFIG #
DAY = "13"
TEST_SOLUTION_PART_1 = 0
TEST_SOLUTION_PART_2 = 0

In [56]:
#%%time
# DAY 13 #

def run(part,i):

    def clean_input(input):
        clean = []
        return clean

    # Start here #
    input = clean_input(i)

    return 1

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test failed. Expected 0, but got 1.
Test failed. Expected 0, but got 1.
Part 1: 1
Part 2: 1
